In [215]:
data_amazon = open('data/amazon_cells_labelled.txt', 'rb')
data_imdb = open('data/imdb_labelled.txt', 'rb')
data_yelp = open('data/yelp_labelled.txt', 'rb')
data = [data_amazon, data_imdb, data_yelp]

In [216]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
lancaster_stemmer = LancasterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()


sentences = []
labels = []
stop_words = [str(w) for w in stopwords.words('english')]
tokenizer = RegexpTokenizer(r'\w+')

def preprocess(sentence):
    result = tokenizer.tokenize(sentence.lower())
    result = [w for w in result if w not in stop_words]
    return result

for d in data:
    raw_text = ''.join([row for row in d])
    sentences.extend(map(preprocess, re.split('\t[0-1]\n', raw_text))[:-1])
    labels.extend(map(lambda x: int(x[1]), re.findall('\t[0-1]\n', raw_text)))

care


In [261]:
# Splitting dataset

TRAINING_SIZE = 400

train_sentences = []
train_labels = []

test_sentences = []
test_labels = []

for i in range(3):
    n_positive = 0
    n_negative = 0
    for j in range(i * 1000, i * 1000 + 1000):
        if labels[j] == 1:
            if n_positive < (TRAINING_SIZE):
                train_sentences.append(sentences[j])
                train_labels.append(labels[j])
                n_positive += 1
            else:
                test_sentences.append(sentences[j])
                test_labels.append(labels[j])
        else:
            if n_negative < (TRAINING_SIZE):
                train_sentences.append(sentences[j])
                train_labels.append(labels[j])
                n_negative += 1
            else:
                test_sentences.append(sentences[j])
                test_labels.append(labels[j])

In [287]:
corpus = {}
n_words = 0
for sentence in train_sentences:
    for word in sentence:
        if word not in corpus:
            corpus[word] = n_words
            n_words += 1

In [325]:
import numpy as np

def extract_features_bog(sentences, corpus):
    xs = np.zeros((len(sentences), len(corpus)), dtype=float)
    for i in range(len(sentences)):
        for j in range(len(sentences[i])):
            if sentences[i][j] in corpus:
                xs[i][corpus[sentences[i][j]]] += 1
    return xs
    
def postprocess(features):
    for f in features[:5]:
        for j in range(f.shape[0]):
            # log-normalization
            f[j] = np.log(f[j] + 1)

train_bow_xs = extract_features_bog(train_sentences, corpus)
test_bow_xs = extract_features_bog(test_sentences, corpus)

postprocess(train_bow_xs)
postprocess(test_bow_xs)

In [403]:
def pca(features):
    print features.shape
    U, D, V = np.linalg.svd(features, full_matrices=False)
    print U.shape
    print D.shape
    print V.shape
    
    dims = [10, 50, 100]
    
    reconstruced = []
    
    for dim in dims:
        result = D[0] * U[:, 0] * V[0, :]
        for i in range(1, dim):
            result += D[i] * U[:, i] * V[i, :]
        reconstruced.append(result)
    return reconstruced

(2400, 7407)
(2400, 2400)
(2400,)
(2400, 7407)
